# Problème TwoEq6

In [ ]:
using ForwardDiff, JuMP, Ipopt

In [ ]:
include("../src/enlsip_functions_castor2.jl")

In [ ]:
n = 2
m = 2
nb_eq = 0
nb_constraints = 2

In [ ]:
# Résidus

function r(x::Vector)
    return [x[1]/(1-x[1]) - 5*log(0.4*(1-x[1])/x[2]) + 4.45977;
        x[2] - 0.4 + x[1]/2]
end
resTwoeq6 = ResidualsEval(0)

function (resTwoeq6::ResidualsEval)(x::Vector, rx::Vector, J::Matrix)
    
    if resTwoeq6 == 1
        rx[:] = [x[1]/(1-x[1]) - 5*log(0.4*(1-x[1])/x[2]) + 4.45977;
        x[2] - 0.4 + x[1]/2]
        
    elseif resTwoeq6 == 2
        J[:] = ForwardDiff.jacobian(r,x)
    end
    return
end

In [ ]:
# Contraintes 

function c(x::Vector)
    return [x[1] - 0.0001; x[2] - 0.0001]
end

consTwoeq6 = ConstraintsEval(0)

function(consTwoeq6::ConstraintsEval)(x::Vector, cx::Vector, A::Matrix)
    
    if consTwoeq6.ctrl == 1
        cx[:] =  [x[1] - 0.0001; x[2] - 0.0001]
    elseif consTwoeq6.ctrl == 2
        A[:] = ForwardDiff.jacobian(c,x)
    end
    return
end

## Résolution avec ENLSIP-Julia

In [ ]:
x0 = [.6,.1]  # [.9, .5] # 

e = eps(Float64)
se = sqrt(e)
enlsipTwoeq6 = enlsip(x0,resTwoeq6,consTwoeq6,n,m,nb_eq,nb_constraints,ε_abs = e, ε_rel = se, ε_x = se, ε_c = se)

## Résolution avec Ipopt 

In [ ]:
x_low = [.0001, .0001]
x_up = [0.9999, Inf]

# x0 = [.6,.1]   
x0 = [.9, .5] 

model = Model(Ipopt.Optimizer)

@variable(model, x_low[i] <= x[i=1:n] <= x_up[i], start = x0[i])

@NLobjective(model, Min, (x[1]/(1-x[1]) - 5*log(0.4*(1-x[1])/x[2]) + 4.45977)^2 + (x[2] - 0.4 + x[1]/2)^2)

optimize!(model)

value.(model[:x])